In [3]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
import logging

# Visulatisation
%matplotlib widget
import matplotlib.pyplot as plt

import set_path
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.plot_utils as plot_utils

import open3d as o3d
from pyntcloud import PyntCloud
import pandas as pd
from skimage import feature
from scipy import ndimage

from skimage.transform import probabilistic_hough_line, hough_line, hough_line_peaks

In [6]:
# Load data.
tilecode = '2386_9699'

# Labelled LAS file (see Notebook 0).
las_file = '../datasets/Valeriusplein/pointcloud/reduced_' + tilecode + '.laz'

# AHN data folder.
ahn_data_folder = '../datasets/Valeriusplein/ahn/'
# File with BGT building polygons.
bgt_building_file = '../datasets/Valeriusplein/bgt/bgt_buildings.csv'
# File with BGT road polygons.
bgt_road_file = '../datasets/Valeriusplein/bgt/bgt_roads.csv'
# File with <x,y> coordinates of pole-like objects.
bgt_pole_file = '../datasets/Valeriusplein/bgt/custom_poles.csv'
# File with <x,y> coordinates of street furniture objects.
bgt_street_furniture_file = '../datasets/Valeriusplein/bgt/bgt_street_furniture.csv'

# AHNReader for elevation data.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [7]:
plot_utils.plot_bgt(tilecode, bgt_building_file, bgt_road_file, bgt_pole_file, bgt_street_furniture_file, padding=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### **Cable Extraction:** 2D projection

In [8]:
pointcloud = las_utils.read_las(las_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

# mask for unlabelled
mask = np.where(labels == 0)[0]

In [10]:
# 1. Voxelize


# Voxelize point cloud
voxel_size = 0.20
cloud = PyntCloud(pd.DataFrame(points[mask,:], columns=['x','y','z']))
voxelgrid_id = cloud.add_structure("voxelgrid", size_x=voxel_size, size_y=voxel_size, size_z=voxel_size, regular_bounding_box=False)
voxelgrid = cloud.structures[voxelgrid_id]
print('Voxels per axis:',voxelgrid.x_y_z)


Voxels per axis: [250 232  90]


In [11]:
voxelgrid.shape

[0.2, 0.2008620689654169, 0.20222222222222216]

In [12]:
# Unique voxel number for each point in the point cloud
voxelgrid.voxel_n

# Corresponding voxel per axis for each point in the point cloud
voxelgrid.voxel_x, voxelgrid.voxel_y, voxelgrid.voxel_z

# non-empty 26 neighbors of voxel
voxelgrid.get_voxel_neighbors(874)

# voxel centers by index
voxelgrid.voxel_centers

# Query point coords to voxel
voxelgrid.query(points[mask,:][:10])

# voxel grid values?
voxelgrid.segments

# Binary of cell is empty or not
voxelgrid.get_feature_vector().shape

(250, 232, 90)

In [13]:
# 2. Top-down projection
voxelgrid_2d = voxelgrid.get_feature_vector().sum(axis=2)
voxelgrid_2d = np.asarray(voxelgrid_2d > 0, dtype=int)
x, y = voxelgrid_2d.nonzero()

In [20]:
# Proejction Filters
def culling(grid_projection, low_threshold=0, high_threshold=5):
    '''
    An empty space around cables is assumed, therefore dense cells surrounded with
    too many further dense cells can be concluded not being part of a cable. 
    Remove points with no or 3+ 8-neighbors.
    '''
    k = np.array([[1,1,1],[1,0,1],[1,1,1]])
    cell_density = ndimage.convolve(grid_projection, k) * grid_projection
    return np.logical_and(cell_density < high_threshold, cell_density > low_threshold).astype(int)

def closing(grid_projection, n_iterations=1):
    '''
    Removes small holes in the foreground (dilation→erosion)
    '''
    return ndimage.binary_closing(grid_projection, iterations=n_iterations)

def canny_edge(grid_projection, sigma=0.5, low_threshold=0, high_threshold=1):
    '''
    Edge extraction algorithm (can also use openCV implementation)
    '''
    return feature.canny(voxelgrid_2d, sigma=sigma, low_threshold=low_threshold, high_threshold=high_threshold)

In [ ]:
def apply_filters(grid_projection, filters):
    for filter in filters:
        if filter == ''

In [14]:
# 3. High-point density filter / vegation filter

# 3.1.1 Culling
# An empty space around cables is assumed, therefore dense cells surrounded with too many further dense cells can be concluded not being part of a cable. Remove points with no or 3+ 8-neighbors.


# 3.1.2 Closing (morphology)
# 


# 3.3 Canny Filter



In [19]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(voxelgrid_2d, cmap='gray')
ax[0].set_title('Original')
ax[1].imshow(voxelgrid_2d_culling, cmap='gray')
ax[1].set_title('Culling')
ax[2].imshow(voxelgrid_2d_closed, cmap='gray')
ax[2].set_title('Closing')
ax[3].imshow(voxelgrid_2d_edge, cmap='gray')
ax[3].set_title('Canny')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Canny')

In [16]:
print('points in gird:',voxelgrid_2d_culling.sum())

points in gird: 1532


In [332]:
y, x = voxelgrid_2d_culling.nonzero()
eps = 5
line_points = []
non_line_points = []
residuals = []
slope = np.tan(angle + np.pi/2)

for i in range(len(x)):
    residual = abs((y0 - y[i]) - slope * (x0 - x[i]))
    residuals.append(residual)
    if residual < eps:
        line_points.append((x[i],y[i]))
    else:
        non_line_points.append((x[i],y[i]))

len(line_points)

101

In [333]:
voxelgrid_2d_culling[np.array(line_points)[:,1], np.array(line_points)[:,0]] = 0

In [11]:
# 4. Line detection: Hough Transfrom
min_cable_length_m = 5
lines = probabilistic_hough_line(voxelgrid_2d_culling, threshold=10, line_length=int(min_cable_length_m/voxel_size), line_gap=4)

# Generating figure 2
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(voxelgrid_2d_culling, cmap='gray')
ax[0].set_title('Input image')

ax[1].imshow(voxelgrid_2d_culling * 0)
for line in lines:
    p0, p1 = line
    ax[0].plot((p0[0], p1[0]), (p0[1], p1[1]))
    ax[1].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[1].set_xlim((0, voxelgrid_2d_culling.shape[1]))
ax[1].set_ylim((voxelgrid_2d_culling.shape[0], 0))
ax[1].set_title('Probabilistic Hough')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Probabilistic Hough')

In [397]:
fig, ax = plt.subplots(1, sharex=True, sharey=True)
ax.imshow(voxelgrid_2d_culling, cmap='gray')

p0, p1 = lines[0]
ax.plot((p0[0], p1[0]), (p0[1], p1[1]))
p0, p1 = lines[1]
ax.plot((p0[0], p1[0]), (p0[1], p1[1]))
p0, p1 = lines[2]
ax.plot((p0[0], p1[0]), (p0[1], p1[1]))

lines[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((129, 225), (39, 2))

In [44]:
line_properties = []
for s in range(len(lines)):
    p0, p1 = lines[s]
    slope =  (p1[1] - p0[1]) / (p1[0]-p0[0])
    b =  p0[1] - (p0[0] * slope)
    line_properties.append((slope,b))
line_properties = np.asarray(line_properties)

In [ ]:
from itertools import combinations
i=0
for (line_a, line_b) in combinations(list(range(len(lines))), 2):
    slope_diff, b_diff = abs(line_properties[line_a] - line_properties[line_b])
    if slope_diff < 0.1 and b_diff < 3:
        print((line_a, line_b), slope_diff, b_diff)
        i += 1
        fig, ax = plt.subplots(1, sharex=True, sharey=True)
        ax.imshow(voxelgrid_2d_culling, cmap='gray')

        p0, p1 = lines[line_a]
        ax.plot((p0[0], p1[0]), (p0[1], p1[1]))
        p0, p1 = lines[line_b]
        ax.plot((p0[0], p1[0]), (p0[1], p1[1]))

In [385]:

eps = 5

line_points = {}
for s in range(len(lines)):
    line_points[s] = []
line_points['unlabelled'] = []

y, x = voxelgrid_2d_culling.nonzero()
for i in range(len(x)):
    for s in range(len(lines)):
        p0, p1 = lines[s]
        dist_line = np.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2)

        dist_A = np.sqrt((p0[0] - x[i])**2 + (p0[1] - y[i])**2)
        dist_B = np.sqrt((p1[0] - x[i])**2 + (p1[1] - y[i])**2)

        if abs(dist_line - (dist_A + dist_B)) < eps:
            line_points[s].append((x[i],y[i]))
            break

    line_points['unlabelled'].append((x[i],y[i]))



In [ ]:
# 5. Cable extraction: 3D-RANSAC

